In [1]:
from pathlib import Path
path = Path('/data/DIGITANIE_v4/normalisation_stats.npy')
import numpy as np
stats = np.load(path, allow_pickle=True).item()

In [2]:
stats['HELSINKI'.title()]

{'mean': array([4677.6298598 , 4951.11973206, 4950.21863196, 4637.4914173 ]),
 'std': array([2577.73259758, 2590.33073603, 2470.59660255, 2556.43454128]),
 'perc_2': array([612., 825., 991., 593.]),
 'perc_98': array([8110., 8430., 8329., 8333.]),
 'mean_clip': array([4653.96637296, 4931.24502525, 4934.74545525, 4610.70596437]),
 'std_clip': array([2521.03195555, 2541.30805596, 2431.12861806, 2481.652928  ]),
 'perc_995': array([ 9959., 10034.,  9605., 10506.]),
 'mean_clip995': array([4669.47801968, 4943.3877924 , 4943.34277817, 4628.81965565]),
 'std_clip995': array([2555.87816219, 2570.52869772, 2453.67996389, 2530.5947061 ])}

In [6]:
from dl_toolbox.transforms.utils import stretch_to_minmax
import torch
mins = torch.Tensor(stats['HELSINKI'.title()]['perc_2'])
maxs = torch.Tensor(stats['HELSINKI'.title()]['perc_995'])
mean = torch.Tensor(stats['HELSINKI'.title()]['mean_clip995'])
print(stretch_to_minmax(mean, mins, maxs))

tensor([0.4341, 0.4472, 0.4588, 0.4071])


In [4]:
from dl_toolbox.transforms.utils import stretch_to_minmax
mins = torch.Tensorstats['HELSINKI'.title()]['perc_2']
maxs = stats['HELSINKI'.title()]['perc_995']
mean = stats['HELSINKI'.title()]['mean_clip995']
print(stretch_to_minmax(mean, mins, maxs))

TypeError: clip() received an invalid combination of arguments - got (numpy.ndarray, int, int), but expected one of:
 * (Tensor input, Tensor min, Tensor max, *, Tensor out)
 * (Tensor input, Number min, Number max, *, Tensor out)


In [ ]:
stats.keys()

In [ ]:
from pathlib import Path
from itertools import product
data_path = Path('/data')
from dl_toolbox.utils import get_tiles
def paths(num):
    img_dir = data_path/'SemCity-Toulouse-bench/img_multispec_05/TLS_BDSD_RGB'
    msk_dir = data_path/'SemCity-Toulouse-bench/semantic_05/TLS_indMap'
    return img_dir/f'TLS_BDSD_RGB_{num:02}.tif', msk_dir/f'TLS_indMap_{num:02}_1.tif'
test = [(*paths(num), t) for num in [6,11] for t in get_tiles(3504, 3452, 512)]
val = [(*paths(7), t) for t in get_tiles(3504, 3452, 512)]
train = [[*paths(num), t] for num in set(range(1, 17))-{6,7,11} for t in get_tiles(3504, 3452, 876, 863)]
[list(t) for t in zip(*train)]

In [ ]:
list(train)

In [9]:
import torch
a = torch.randint(0,5, size=(1,10,10))
b = torch.nn.functional.one_hot(a.unsqueeze(1), 5).transpose(1,-1).squeeze(-1)
b.shape

torch.Size([1, 5, 10, 10])

In [27]:
import pandas as pd
stats = pd.read_csv('/data/outputs/toulouse_sup_building/2023-10-25_171203/checkpoints/epoch_099/stats.csv', index_col=0)
stats.sort_values('avg_cert', ascending=False, inplace=True)

In [29]:
stats

,msk,img,win,avg_cert
1424,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,29184_36864_512_512,1.000000
1670,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,29184_39424_512_512,1.000000
258,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,24064_17408_512_512,1.000000
225,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,24576_16896_512_512,1.000000
328,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,24576_18432_512_512,1.000000
...,...,...,...,...
898,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,32256_26624_512_512,0.965584
928,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,31744_27136_512_512,0.965429
835,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,31744_25600_512_512,0.963051
796,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,DIGITANIE_v4/TOULOUSE/TOULOUSE_20130924_T_16bi...,17408_25088_512_512,0.962089
